
# Reliability of 2D Embeddings with UMAP and scDEED-inspired Method

In this lesson, we will explore how to evaluate the reliability of 2D embeddings obtained with UMAP, 
using an approach inspired by the **scDEED** algorithm.  
We will apply it to the **Gamma-Ray Burst (GRB) latent-space datasets**. 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances
from scipy.stats import pearsonr
import umap.umap_ as umap


## Step 1: Load the GRB Latent-Space Dataset

We use pre-computed GRB latent-space vectors (30-dimensional). These are the products of three Convolutional Autoencoders, each one applied to a specific Waterfall plot dimension.


In [ ]:
# Load GRB latent space datasets (provided separately as .npy files)
hl_1 = np.load('scdeed_data/short_timescales.npy')
hl_2 = np.load('scdeed_data/medium_timescales.npy')
hl_3 = np.load('scdeed_data/long_timescales.npy')

# Normalize each part
def normalize(x):
    return (x - np.min(x)) / (np.max(x) - np.min(x))

hl_1, hl_2, hl_3 = normalize(hl_1), normalize(hl_2), normalize(hl_3)

# Concatenate into final latent space
dataset = np.concatenate((hl_1, hl_2, hl_3), axis=1)

print("Number of GRBs:", dataset.shape[0])
print("Dimensionality before UMAP:", dataset.shape[1])



## Step 2: Apply UMAP for Dimensionality Reduction

We now reduce the latent space (30D) into **2D** using UMAP.  
This makes the data easier to visualize and explore.  

In [ ]:
# UMAP parameters
NN = 30
MD = 0.0
LR = 0.001
NEPOCHS = 1000
LOC_CON = 0.5

reducer = umap.UMAP(n_neighbors=NN, min_dist=MD, n_components=2,
                    metric='euclidean', n_epochs=NEPOCHS,
                    local_connectivity=LOC_CON, learning_rate=LR,
                    random_state=42)
X_latent = reducer.fit_transform(dataset)

plt.figure(figsize=(8,6))
plt.scatter(X_latent[:,0], X_latent[:,1], s=10, alpha=0.7)
plt.title("UMAP 2D Embedding of GRB Latent Space")
plt.xlabel("UMAP-1")
plt.ylabel("UMAP-2")
plt.show()

# How can we assess the reliability of the position of each event in the final distribution?

UMAP is a powerful algorithm which reduces the dimensionality of a dataset exploiting complex and non-linear relations between events. When used to generally look at a low dimensional distribution, without caring for the position of specific events, the lack of interpretability is usually not an issue. In our case, where we want to investigate the properties and the location of specific GRBs, this could represent an obstacle. 
Let's try to understand how we can improve the reliability of the final embedding.


## Step 3: Reliability Assessment step by step

We now proceed to:  
- Compute reliability scores (Pearson correlation of neighbor distances).  
- Generate null distributions by permuting the dataset.  
- Visualize real vs null distributions.  


Firstly, we begin by evaluating the correlation of the distances of the closest k events in the original and embedded space, calculated in the embedded space.

In [ ]:
# We define the "neighborhood" of an event.
n_samples = dataset.shape[0]
k = int(n_samples/2)

# We evaluate the distance between events in the original space and in the embedded space.
dist_orig = pairwise_distances(dataset)
dist_latent = pairwise_distances(X_latent)

# In the following cicle, for each event, we evaluate the correlation of the k nearest neighbors in the original and embedded space.
correlation_vector = np.zeros(n_samples) # One value for each event 
for i in range(n_samples):
    # For the ith event, we identify the closest k events in the original space...
    closest_orig_indices = np.argsort(dist_orig[i])[1:k+1]
    # ... and the closest k in the embedded space
    closest_latent_indices = np.argsort(dist_latent[i])[1:k+1]
    # We evaluate the distances of these events in the final embedded space
    closest_orig_distances = dist_latent[i][closest_orig_indices]
    closest_latent_distances = dist_latent[i][closest_latent_indices]
    # And we find the Pearson correlation between the distance vectors
    correlation, _ = pearsonr(closest_orig_distances, closest_latent_distances)
    correlation_vector[i] = correlation

We have now correlation values: to what we can compare them? We need a null distribution!

In [ ]:
# One distribution is not statistically significant enough, we need to compute it several times.
num_permutations = 3
# We define our final vector: we will fill it later
null_distributions = np.zeros((num_permutations, n_samples))

# We loop through number of permutations
for perm in range(num_permutations):
    
    # Permute features: we create a dataset with no meaningfull neighborhood structure
    permuted_original = np.zeros_like(dataset)
    for j,row in enumerate(dataset):
        iii = np.random.permutation(dataset.shape[1])
        permuted_original[j] = row[iii]
    # We generate the UMAP embedding of the permuted dataset
    reducer = umap.UMAP(n_neighbors=NN, min_dist=MD, n_components=2,
                        metric='euclidean', n_epochs=NEPOCHS, local_connectivity=LOC_CON,
                        learning_rate=LR, random_state=perm)
    permuted_latent = reducer.fit_transform(permuted_original)
    
    # We calculate the same correlation we evaluated for the actual dataset, but ths time for the permuted one
    dist_orig = pairwise_distances(permuted_original)
    dist_latent = pairwise_distances(permuted_latent)
    for i in range(n_samples):
        closest_orig_indices = np.argsort(dist_orig[i])[1:k+1]
        closest_latent_indices = np.argsort(dist_latent[i])[1:k+1]
        null_distributions[perm,i], _ = pearsonr(dist_latent[i][closest_orig_indices],
                                                 dist_latent[i][closest_latent_indices])
        
# We flatten the null distribution: we performed more than one permutation only for statistical reasons.
null_distributions = null_distributions.flatten()

Now that we have a correlation vector and a null distribution of correlations, we can assess the reliability of each event, by comparing its correlation value to the null distribution. 
- **Green** = Trustworthy (above 95th percentile).  
- **Red** = Dubious (below 5th percentile).  
- **Gray** = Neither (in between).  

In [ ]:
# We evaluate the 95th and 5th percentile of the null distribution.
percentile_95 = np.percentile(null_distributions, 95)
percentile_5 = np.percentile(null_distributions, 5)

# We plot the null and the actual correlation distributions.
bins = 150
plt.figure(figsize=(8,6))
plt.hist(null_distributions, bins=bins, density=True, histtype='step',
         color='black', linewidth=2, label="Null distribution")
plt.hist(correlation_vector, bins=bins, density=True, histtype='step',
         color='blue', linewidth=2, label="Real embedding")
plt.axvline(percentile_5, color='red', linestyle='--', label="5th percentile")
plt.axvline(percentile_95, color='green', linestyle='--', label="95th percentile")
plt.xlabel("Pearson Correlation")
plt.ylabel("Density")
plt.legend()
plt.title("Real vs Null Reliability Distributions")

# We define trustworthy and dubious events, as previously described.
significativity = (correlation_vector >= percentile_95).astype(int)
dub = (correlation_vector <= percentile_5)
significativity[dub] = -1


# Plot the results!
colors = {1: 'green', 0: 'gray', -1: 'red'}
labels = {1: 'Trustworthy', 0: 'Unlabeled', -1: 'Dubious'}

plt.figure(figsize=(8,6))
for val, color in colors.items():
    idx = significativity == val
    plt.scatter(X_latent[idx,0], X_latent[idx,1], s=20, alpha=0.7, c=color, label=labels[val])
plt.xlabel("UMAP-1")
plt.ylabel("UMAP-2")
plt.title("GRB Embedding Reliability")
plt.legend()


plt.show()


## 📝 Exercises

1. Change the UMAP parameters (or change the whole dimensionality reduction technique!) and re-run the notebook.  
   - How does the number of dubious points change?  
   - How does the overall structure of the embedding change?  

3. Reflect on the physical meaning:  
   - Why is it important to identify dubious embeddings in GRB latent space?  
   - How might misleading embeddings affect astrophysical interpretations?  
